In [ ]:
using DifferentialEquations
using Plots

In [3]:
import Pkg;


    Updating registry at `C:\Users\olarinoyem\.julia\registries\General.toml`


   Resolving package versions...

   Installed TreeViews ─────────────── v0.3.0
   Installed Sundials_jll ──────────── v5.2.1+0


   Installed CEnum ─────────────────── v0.4.2
   Installed BoundaryValueDiffEq ───── v2.9.0
   Installed DifferentialEquations ─── v7.6.0
   Installed Distances ─────────────── v0.10.7


   Installed KrylovKit ─────────────── v0.5.4


...

  ✓ LinearSolve


  ✓ SteadyStateDiffEq


  ✓ DiffEqNoiseProcess


  ✓ OrdinaryDiffEq


  ✓ DelayDiffEq


In [1]:
function sir(;s0,  beta, gamma, r0, title="Epidemic simulation using the SIR model")
    f(r,p,t) = gamma * (1 - r - s0 * exp(-beta * r / gamma))

    tspan = (0.0,60.0)
    prob = ODEProblem(f,r0,tspan)
    r = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)   # Fifth order Tsitouras method
    s = s0 * exp.( -beta .* r ./ gamma)
    x = 1 .- s .- r

    plot(r,linewidth=5,title=title,
         xaxis="Time (t)",yaxis="% of population)",label="Recovered") # legend=false
    plot!(r.t, s,lw=3,ls=:dash,label="Susceptible")
    plot!(r.t, x,lw=3,label="Infected")
end

sir(s0 = 0.99, beta = 1.0, gamma = 0.2, r0 = 0.0001)

UndefVarError: UndefVarError: ODEProblem not defined